In [2]:
import pulp
import sys
import preprocess_data
import GA
import pandas as pd

In [4]:
input_file = "./csv/conference_schedule_mini.csv"
parsed_input_file, papers_schedule = preprocess_data.parse_csv_to_array(input_file)
session_durations, num_tracks_per_session = preprocess_data.session_details(parsed_input_file)
session_details = [(session_durations[i], num_tracks_per_session[i]) for i in range(len(session_durations))]
print(session_details)

[(30.0, 2), (60.0, 2), (60.0, 2), (15.0, 1), (60.0, 1), (60.0, 2), (60.0, 1)]


In [8]:
papers_dict = preprocess_data.create_papers(input_file)
papers = papers_array = [x for x in papers_dict.values()]

for item in papers:
    print(item.print_paper())

ID: 1, Authors: ['Jürgen Cito', ' Jiasi Shen', ' Martin C. Rinard', '  Pre-print', '  Media Attached'], Duration: 6.0, Topic: 1
ID: 2, Authors: ['Jason Tsay', ' Alan Braz', ' Martin Hirzel', ' Avraham Shinnar', ' Todd Mummert', '  Pre-print', '  Media Attached'], Duration: 6.0, Topic: 1
ID: 3, Authors: ['Timofey Bryksin', ' Victor Petukhov', ' Ilya Alexin', ' Stanislav Prikhodko', ' Alexey Shpilman', ' Vladimir Kovalenko', ' Nikita Povarov', '  Pre-print', '  Media Attached'], Duration: 6.0, Topic: 1
ID: 4, Authors: ['Tomoki Nakamaru', ' Tomomasa Matsunaga', ' Tetsuro Yamazaki', ' Soramichi Akiyama', ' Shigeru Chiba', '  Pre-print', '  Media Attached'], Duration: 6.0, Topic: 1
ID: 5, Authors: ['Nan Yang', ' Pieter Cuijpers', ' Ramon Schiffelers', ' Johan Lukkien', ' Alexander Serebrenik', '  Media Attached'], Duration: 6.0, Topic: 1
ID: 6, Authors: ['Leonardo Da Silva Sousa', ' Diego Cedrim', ' Alessandro Garcia', ' Willian Oizumi', ' Ana Carla Bibiano', ' Daniel Oliveira', ' Miryung K

In [10]:
num_sessions = len(session_details)
max_tracks = max(num_of_tracks for _, num_of_tracks in session_details)
num_papers = len(papers)

In [13]:
# Set up the problem
prob = pulp.LpProblem("Conference_Schedule", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("paper_schedule", 
                          [(i, j, k) for i in range(num_papers)
                                     for j in range(num_sessions)
                                     for k in range(max_tracks)],
                          cat='Binary')

# Objective function - As before, we want to minimize the total number of slots used
prob += pulp.lpSum(x[i, j, k] for i in range(num_papers) for j in range(num_sessions) for k in range(max_tracks))

# Constraints

# Constraint 1: Total duration of papers in a session cannot exceed session duration
for j, (session_duration, num_of_tracks) in enumerate(session_details):
    for k in range(num_of_tracks):
        prob += pulp.lpSum(x[i, j, k] * papers[i].duration for i in range(num_papers)) <= session_duration


In [14]:
def have_common_authors(paper1, paper2):
    return not set(paper1.authors).isdisjoint(paper2.authors)


In [15]:
for i in range(num_papers):
    for j in range(i+1, num_papers):
        if have_common_authors(papers[i], papers[j]):
            for session in range(num_sessions):
                for track_i in range(session_details[session][1]):
                    for track_j in range(session_details[session][1]):
                        if track_i != track_j:
                            prob += x[i, session, track_i] + x[j, session, track_j] <= 1


# Constraint 3: Papers with the same topic should be scheduled in the same session
for topic in set(paper.topic for paper in papers):
    for j in range(num_sessions):
        # For each topic, find papers with that topic and ensure they are scheduled in the same session
        topic_papers = [i for i, paper in enumerate(papers) if paper.topic == topic]
        for i in topic_papers:
            for k in range(session_details[j][1]):
                prob += pulp.lpSum(x[i, j, m] for m in range(session_details[j][1])) == x[i, j, k]

# Solve the problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/myenv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/5k/15bqvj117x74tp3qhvsy1m880000gn/T/f6035d7088684bc799fb8158193d07c6-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/5k/15bqvj117x74tp3qhvsy1m880000gn/T/f6035d7088684bc799fb8158193d07c6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4614 COLUMNS
At line 15511 RHS
At line 20121 BOUNDS
At line 20598 ENDATA
Problem MODEL has 4609 rows, 476 columns and 9094 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0002I 272 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 t

1

In [16]:
# Output the results
for v in prob.variables():
    if v.varValue == 1:
        print(v.name, "=", v.varValue)